In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup 
import yahoo_finance

In [2]:
def produce_soup_from_url(url : str):

    r = requests.get(url)
    
    return BeautifulSoup(r.content,'html5lib')

In [3]:
def dataframe_builder(profile_dict : dict):

    return pd.DataFrame({'practices' : profile_dict['practices'], 'services' : profile_dict['services']})

    


In [4]:
def df_to_csv(df : pd.DataFrame, filename : str):
    # df.to_csv(path_or_buf=r'...\database')
    return df.to_csv(path_or_buf=fr'..\..\database\{filename}.csv', index=False)

In [28]:
# BetterGov URL : https://www.bettergov.co.uk/

profile_dict = {'Practises_URL': [r'https://www.bettergov.co.uk/'], 'Practises': [], 'Services_URL': [], 'Services': []}

soup = produce_soup_from_url(profile_dict['Practises_URL'][0])
html = soup.find_all(lambda tag: tag.name == 'a' and tag.has_attr('href') and r'https://www.bettergov.co.uk/better-' in tag['href'] and tag.find('span'))
html = list(set(html))

for row_i in html:
    
    service_url = row_i['href']
    # profile_dict['Services_URL'].append(service_url)

    service_soup = produce_soup_from_url(service_url)

    services = service_soup.find_all('h5')[1::]
    
    for row_j in services:
        profile_dict['Services'].append(row_j.text)
        profile_dict['Practises'].append(row_i.find('span', attrs = {'class': "nectar-menu-label nectar-pseudo-expand"}).text)
        profile_dict['Services_URL'].append(service_url)


# dataframe_builder(profile_dict)

In [32]:
5*[1]

[1, 1, 1, 1, 1]

In [34]:
for k,v in profile_dict.items():
    print(f'{k} ---- length: {len(v)}')

Practises_URL ---- length: 28
Practises ---- length: 28
Services_URL ---- length: 28
Services ---- length: 28


In [6]:
# Cambridge Consultants URL : https://www.cambridgeconsultants.com/


profile_dict = {'practises_url': ['https://www.cambridgeconsultants.com/'], 'practises': [], 'services_url': [], 'services': []}

soup = produce_soup_from_url(r'https://www.cambridgeconsultants.com/')

filtered_html = soup.find_all('ul', attrs = {'id' : 'menu-deep-tech'})[0].select('a') # extracts all rows with links correspondong to dropdown menu "deep tech"

for row_i in filtered_html:

    service_url = row_i['href']
    profile_dict['services_url'].append(service_url)
    # profile_dict['practises'].append(row.text)

    service_soup = produce_soup_from_url(service_url)

    # service_soup.find_all('div', attrs = {'class' : "et_pb_row et_pb_row_15"})

    raw_html = service_soup.find_all('ul', attrs = {'class' : "et_pb_tabs_controls clearfix"})[0].select('li') # using the initial attrs means we can access the specific children we need to as

    for row_j in raw_html:

        profile_dict['practises'].append(row_i.text)
        profile_dict['services'].append(row_j.text)




In [32]:

df = dataframe_builder(profile_dict)
df_to_csv(df,filename='cambridge_consultants')

In [60]:
# Capco URL: https://www.capco.com/Services

profile_dict = {'practises_url': ['https://www.capco.com'], 'practises': [], 'services_url': [], 'services': []}

soup = produce_soup_from_url(r'https://www.capco.com')
services_html = soup.find_all(lambda tag: tag.name == 'a' and '/Services/' in tag['href'] )

for row_i in services_html:

    service_url = profile_dict['practises_url'][0] + row_i['href']
    # print(service_url)
    profile_dict['services_url'].append(service_url)

    service_soup = produce_soup_from_url(service_url)


    exclude_list = ['/Services/digital/knowable','/Services/digital/Further-Swiss-Solutions']
    services_list = []
    # service_soup = produce_soup_from_url(profile_dict['services_url'][0])


    html = service_soup.find_all('div', attrs = {'class' : 'article-content'})

    for row_j in html:

        if row_j.find('h2'):
            if row_j.find('a')['href'] not in exclude_list:
                services_list.append(row_j.find('h2').text)
                # profile_dict['practises'].append(row_i.text)

    services_list = list(set(services_list))
    profile_dict['practises'] += [row_i.text]*len(services_list)
    # print([row_i.text]*len(services_list))


    profile_dict['services'] += services_list
    # print(profile_dict)

In [63]:
df = dataframe_builder(profile_dict)
df_to_csv(df, 'capco')

In [41]:
# Cognizant Technology Solutions Corp

profile_dict = {'practises_url': ['https://www.cognizant.com'], 'practises': [], 'services_url': [], 'services': []}

soup = produce_soup_from_url(r'https://www.cognizant.com')
practises_html = soup.find_all('a', href = lambda href: href and "/uk/en/services/" in href, target = True, class_ = False) #<a class="p-half d-block fw-normal text-white cog-header__megamenu-item" href="/uk/en/services/ai" role="link" aria-label="Data & AI" target="_self" data-cmp-data-layer="{"dropDownMenuTag-c33be6f82d":{"xdm:trackingType":"dropDownMenuTag","xdm:location":"Header","dc:title":"Data & AI","xdm:linkURL":"/content/cognizant-dot-com/uk/en/services/ai"}}" data-cmp-clickable>


for row_i in practises_html:

    profile_dict['services_url'].append(row_i['href'])

    services_soup = produce_soup_from_url(profile_dict['practises_url'][0] + row_i['href'])
    services_html = services_soup.find_all('span', attrs = {'class' : 'cmp-accordion__title'})

    for row_j in services_html:
        
        profile_dict['services'].append(row_j.text)
        profile_dict['practises'].append(row_i.text.strip())


df = dataframe_builder(profile_dict)
df_to_csv(df,filename='cognizant')


/uk/en/services/application-services
/uk/en/services/ai
/uk/en/services/business-process-services
/uk/en/services/cloud-solutions
/uk/en/services/core-application-modernization
/uk/en/services/digital-strategy
/uk/en/services/enterprise-platform-services
/uk/en/services/experience-services
/uk/en/services/infrastructure-services
/uk/en/services/intelligent-process-automation
/uk/en/services/iot-solutions
/uk/en/services/enterprise-quality-engineering-assurance
/uk/en/services/cybersecurity-services
/uk/en/services/software-engineering-services
/uk/en/services/sustainability-services
/uk/en/services/consulting


In [6]:
# Infosys

profile_dict = {'practises_url': ['https://www.infosys.com/services/'], 'practises': [], 'services_url': [], 'services': []}
soup = produce_soup_from_url(r'https://www.infosys.com/services/')
practises_html = soup.find_all('li', attrs = {'class' : "col-lg-4 col-md-4 col-sm-4 col-xs-12"})
practises_list = []

for row_i in practises_html:
    for row_j in row_i.find_all('a'):
        
        service_url = profile_dict['practises_url'][0][:-10] + row_j['href']
        profile_dict['services_url'].append(service_url)

        services_soup = produce_soup_from_url(profile_dict['practises_url'][0][:-10] + row_j['href'])

        soup.find_all('p', attrs = {'class' : 'offering-title'})
        services_html = services_soup.find_all('div', attrs = {'class' : "offerings-row clearfix"})


        for offering in services_html:
            for service in offering.find_all('a'):
                profile_dict['services'].append(service.text.strip())
                profile_dict['practises'].append(row_j.text.strip())

In [7]:
df = dataframe_builder(profile_dict)
df_to_csv(df,filename='infosys')

In [6]:
# IQVIA Holdings Inc


profile_dict = {'practises_url': ['https://www.infosys.com/services/'], 'practises': [], 'services_url': [], 'services': []}
soup = produce_soup_from_url(r'https://www.infosys.com/services/')
practises_html = soup.find_all('h3', attrs = {'class' : "headline"})
practises_html


[]

In [18]:
# Kubrick Group

profile_dict = {'practices_url': [r'https://www.kubrickgroup.com/uk/what-we-do'], 'practices': [], 'services_url': [], 'services': []}
soup = produce_soup_from_url(profile_dict['practices_url'][0])
practices_html_block = soup.find_all('section', attrs = {'id' : "our-practices"})[0]

# Each element of list is a practices' html
practices_html = [div for div in practices_html_block.find_all('div', attrs={'class': 'col-12'}) if div.find('p')]

for practice in practices_html:
    for service in practice.find_all('li'):
        profile_dict['practices'].append(practice.find_all('h3')[0].text)
        profile_dict['services'].append(service.text)
        # print(f"{practice.find_all('h3')[0].text} ------- {service.text}")



NameError: name 'produce_soup_from_url' is not defined

In [135]:
practices_html[0].find_all('li')

[<li>Business Analysis &amp; Business Process Mining</li>,
 <li>Requirements Engineering</li>,
 <li>Agile Product Management</li>,
 <li>Change &amp; Risk Management</li>,
 <li>Data Product Testing &amp; Validation</li>,
 <li>Data Product Launch &amp; Deployment</li>,
 <li>Data Product Lifecycle Management</li>]

In [142]:
practices_html_block = soup.find_all('section', attrs = {'id' : "our-practices"})[0]

# Each element of list is a practices' html
practices_html = [div for div in practices_html_block.find_all('div', attrs={'class': 'col-12'}) if div.find('p')]

for practice in practices_html:
    for service in practice.find_all('li'):
        print(f"{practice.find_all('h3')[0].text} ------- {service.text}")

Data Product Management ------- Business Analysis & Business Process Mining
Data Product Management ------- Requirements Engineering
Data Product Management ------- Agile Product Management
Data Product Management ------- Change & Risk Management
Data Product Management ------- Data Product Testing & Validation
Data Product Management ------- Data Product Launch & Deployment
Data Product Management ------- Data Product Lifecycle Management
Data Engineering ------- Modern Data Architecture & Data Modelling
Data Engineering ------- Data Connectivity and Integration
Data Engineering ------- Cloud Data Warehouse and Lake Development
Data Engineering ------- DataOps
Data Engineering ------- Data Orchestration
Data Engineering ------- Data Streaming​
Data Engineering ------- Scalability & Performance Optimisation
Data Engineering ------- Database Design & Development​
Data & AI Governance ------- Data Governance Frameworks & Policy
Data & AI Governance ------- Policy-as-Code
Data & AI Govern

In [141]:
for practice in practices_html:
    print(practice.find_all('li'))
    for service in practice.find_all('li'):
        print(service.text)
        print(practice)
    #     print(f"{service.find_all('h3')[0].text} ------- {service.text}")

[<li>Business Analysis &amp; Business Process Mining</li>, <li>Requirements Engineering</li>, <li>Agile Product Management</li>, <li>Change &amp; Risk Management</li>, <li>Data Product Testing &amp; Validation</li>, <li>Data Product Launch &amp; Deployment</li>, <li>Data Product Lifecycle Management</li>]
Business Analysis & Business Process Mining
Requirements Engineering
Agile Product Management
Change & Risk Management
Data Product Testing & Validation
Data Product Launch & Deployment
Data Product Lifecycle Management
[<li>Modern Data Architecture &amp; Data Modelling</li>, <li>Data Connectivity and Integration</li>, <li>Cloud Data Warehouse and Lake Development</li>, <li>DataOps</li>, <li>Data Orchestration</li>, <li>Data Streaming​</li>, <li>Scalability &amp; Performance Optimisation</li>, <li>Database Design &amp; Development​</li>]
Modern Data Architecture & Data Modelling
Data Connectivity and Integration
Cloud Data Warehouse and Lake Development
DataOps
Data Orchestration
Data

In [87]:
practices_html[0]

<div class="col-12">
	<div class="accordianstep closed row">
		<div class="top col-12">
			<div class="inner">
				<h3 class="headline">Data Product Management</h3>
				<button class="toggle reset"><svg name="button-circle" viewBox="0 0 96 97" xmlns="http://www.w3.org/2000/svg">
		<use xlink:href="#button-circle"></use>
	</svg></button>
			</div>
		</div>
		<div class="content col-10">
			<p><strong>Leverage the power of end-to-end Agile product development, delivery, and management.</strong></p>
<p>We specialise in crafting data-driven solutions and products tailored to your business needs, ensuring flexibility, rapid iterations, and collaborative development for enhanced decision-making and innovation.<br/></p>
<p>Solutions include:</p>
<ul><li>Business Analysis &amp; Business Process Mining</li><li>Requirements Engineering</li><li>Agile Product Management</li><li>Change &amp; Risk Management</li><li>Data Product Testing &amp; Validation</li><li>Data Product Launch &amp; Deployment</

In [88]:
practices_html[1]

<div class="top col-12">
			<div class="inner">
				<h3 class="headline">Data Product Management</h3>
				<button class="toggle reset"><svg name="button-circle" viewBox="0 0 96 97" xmlns="http://www.w3.org/2000/svg">
		<use xlink:href="#button-circle"></use>
	</svg></button>
			</div>
		</div>

In [60]:
practices_html = soup.find_all('section', attrs = {'id' : "our-practices"})[0].find_all('div', {'class' : 'content col-10'}) # Each element in services html is a practice
practices_html[0].find_all('li')
for practice in practices_html:
    print(practice.find_all('li'))
    print(practice.find_all('h3'))

[<li>Business Analysis &amp; Business Process Mining</li>, <li>Requirements Engineering</li>, <li>Agile Product Management</li>, <li>Change &amp; Risk Management</li>, <li>Data Product Testing &amp; Validation</li>, <li>Data Product Launch &amp; Deployment</li>, <li>Data Product Lifecycle Management</li>]
[]
[<li>Modern Data Architecture &amp; Data Modelling</li>, <li>Data Connectivity and Integration</li>, <li>Cloud Data Warehouse and Lake Development</li>, <li>DataOps</li>, <li>Data Orchestration</li>, <li>Data Streaming​</li>, <li>Scalability &amp; Performance Optimisation</li>, <li>Database Design &amp; Development​</li>]
[]
[<li>Data Governance Frameworks &amp; Policy</li>, <li>Policy-as-Code</li>, <li>Master Data &amp; Reference Data Management</li>, <li>Data Quality Management</li>, <li>Data Cataloging &amp; Lineage</li>, <li>Data Privacy &amp; Compliance</li>, <li>Data Domain Modelling</li>]
[]
[<li>Data Analysis &amp; Insights​</li>, <li>Advanced Data Visualisation</li>, <li>D

In [9]:
from SupportFunctions import get_company_details
import pandas as pd
import json

In [5]:
company_json = get_company_details('INFY')

In [17]:
comp_dict = json.loads(company_json)

pd.DataFrame(comp_dict, index=[0])


,Public/Private,Previous Close,52 Week Range,Sector,Industry,Full Time Employees,Market Cap,Fiscal Year Ends,Revenue,EBITDA
0,Public,17.13,N/A,Technology,Information Technology Services,317240,71729012736,N/A,18562000896,4206000128


In [5]:
get_company_details('Kubrick Group')

404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/KUBRICK%20GROUP?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=KUBRICK+GROUP&crumb=g7Bc4pxK1.P


'{"Public/Private": "Private"}'

In [7]:
import pandas as pd



In [9]:
df = pd.DataFrame({'col1' : [1,2,3]})

In [11]:
print(df.to_markdown())

|    |   col1 |
|---:|-------:|
|  0 |      1 |
|  1 |      2 |
|  2 |      3 |
